## Лабораторна робота 2.1.2

### Завдання: Створити RNN для семантичного аналізу тексту з використанням:  моделі GRU. 
Розробити власну архітектуру, кількість рекурентних шарів, функції
активації, вибрати оптимізот. .
Отримати для двох моделей по дві рецензії (позитивну і негативну).

In [1]:
#Імпортуємо всі необхідні біблідтеки
import os

import numpy as np
import re

from tensorflow.keras.layers import Dense, GRU, Input, Dropout, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
#Задаємо тексти 

#Позитивний текст
texts_true = [
    "I really like this song!",
    "It is the best place I have ever seen.",
    "Honey, you are so beautiful girl.",
]

#Негативний текст
texts_false = [
    "This is a terrible song! Please turn it off.",
    "I am disappointed with this place. It used to be better here.",
    "What a horror! You are a monster.",
]


In [3]:
# Об'єднання текстів
texts = texts_true + texts_false
count_true = len(texts_true)
count_false = len(texts_false)
total_lines = count_true + count_false
print(count_true, count_false, total_lines)

#Токенізація
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(texts)

#Робота з текстом
dist = list(tokenizer.word_counts.items())
print(dist[:10])
print(texts[0][:100])

max_text_len = 10
data = tokenizer.texts_to_sequences(texts)
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)

print(list(tokenizer.word_index.items()))

X = data_pad
Y = np.array([[1, 0]]*count_true + [[0, 1]]*count_false)
print(X.shape, Y.shape)

indices = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
X = X[indices]
Y = Y[indices]

#Створення моделі та навчання моделі

model = Sequential()
model.add(Embedding(maxWordsCount, 64, input_length=max_text_len))
model.add(GRU(50))
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

history = model.fit(X, Y, batch_size=64, epochs=120)

#Функція для перетворення послідовності чисел у текст

reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

def sequence_to_text(list_of_indices):
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)


3 3 6
[('i', 3), ('re', 3), ('lly', 1), ('like', 1), ('this', 3), ('song', 2), ('it', 3), ('is', 2), ('the', 1), ('best', 1)]
I really like this song!
[[ 0  0  0  0  1  2 11 12  3  5]
 [ 6 13 14  7  8  1 15 16 17 18]
 [ 0  0  0 19  9  2 20 10 21 22]
 [ 0  3  6 23  5 24 25 26  4 27]
 [30  3  7  8  4 31 32 10 33 34]
 [ 0  0  0  0 35 36 37  9  2 38]]
[('i', 1), ('re', 2), ('this', 3), ('it', 4), ('song', 5), ('is', 6), ('l', 7), ('ce', 8), ('you', 9), ('be', 10), ('lly', 11), ('like', 12), ('the', 13), ('best', 14), ('h', 15), ('ve', 16), ('ever', 17), ('seen', 18), ('honey', 19), ('so', 20), ('utiful', 21), ('girl', 22), ('terrible', 23), ('le', 24), ('se', 25), ('turn', 26), ('off', 27), ('dis', 28), ('ointed', 29), ('with', 30), ('used', 31), ('to', 32), ('better', 33), ('here', 34), ('wh', 35), ('t', 36), ('horror', 37), ('onster', 38)]
(6, 10) (6, 2)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape        

In [4]:
#Перевірка позитивного тексту
text1 = "I think this is the most beautiful picture in this gallery. ".lower()
data = tokenizer.texts_to_sequences([text1])
data_pad = pad_sequences(data, maxlen=max_text_len)
print(sequence_to_text(data[0]))

res = model.predict(data_pad)
print(res, np.argmax(res), sep='\n')

if np.argmax(res) == 0:
    print("Результат. Текст позитивний.")
else:
    print("Результат. Текст негативний.")

['i', 'this', 'is', 'the', 'be', 'utiful', 'this']
1/1 [==============================] - 1s 534ms/step
[[0.57999444 0.42000556]]
0
Результат. Текст позитивний.


In [5]:
#Перевірка негативного тексту

text2 = "I really hate school.".lower()
data1 = tokenizer.texts_to_sequences([text2])
data1_pad = pad_sequences(data1, maxlen=max_text_len)
print( sequence_to_text(data1[0]) )

res1 = model.predict(data1_pad)
print(res1, np.argmax(res1), sep='\n')


if np.argmax(res1) == 1:
    print("Результат. Текст позитивний.")
else:
    print("Результат. Текст негативний.")

['i', 're', 'lly', 'h']
1/1 [==============================] - 0s 35ms/step
[[0.5880823  0.41191775]]
0
Результат. Текст негативний.


#### Висновок: На практичній роботі я навчилася створити RNN для семантичного аналізу тексту з використанням моделі GRU. Також в ході роботи ця модель була навчана до максимально високої точності